<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/Research-Project-Selected-2/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/mohamedahmedsaadahmed77/OCR-Selected-2.git

Cloning into 'OCR-Selected-2'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 183726 (delta 0), reused 9 (delta 0), pack-reused 183715
Receiving objects: 100% (183726/183726), 64.07 MiB | 19.27 MiB/s, done.
Resolving deltas: 100% (142652/142652), done.
Checking out files: 100% (107101/107101), done.


In [0]:
import numpy as np
import os
from imutils import paths
import cv2
import pandas as pd

def load_images(path):
  print("[INFO] loading images...")
  imagePaths = list(paths.list_images(path))
  imagePaths.sort()
  data = []
  labels = []
  for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.reshape(image, (28, 28, 1))/255.0
    data.append(image)
    labels.append(label)
  data = np.array(data)
  labels=pd.get_dummies(labels)
  labels = np.array(labels)
  print("[INFO] done")
  return data,labels

In [0]:
x_train, y_train = load_images('/content/OCR-Selected-2/Dataset/Training')
x_test, y_test = load_images('/content/OCR-Selected-2/Dataset/Test')

[INFO] loading images...
[INFO] done
[INFO] loading images...
[INFO] done


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.optimizers import RMSprop, Adam

model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation="relu", input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(26, activation="softmax"))
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=128, epochs=8, verbose=2, validation_split=0.1)

Train on 79919 samples, validate on 8880 samples
Epoch 1/8
 - 12s - loss: 0.7823 - accuracy: 0.7667 - val_loss: 11.0791 - val_accuracy: 0.1279
Epoch 2/8
 - 6s - loss: 0.3216 - accuracy: 0.9042 - val_loss: 11.4964 - val_accuracy: 0.1432
Epoch 3/8
 - 6s - loss: 0.2519 - accuracy: 0.9255 - val_loss: 12.4200 - val_accuracy: 0.1392
Epoch 4/8
 - 6s - loss: 0.2156 - accuracy: 0.9360 - val_loss: 14.0605 - val_accuracy: 0.1555
Epoch 5/8
 - 6s - loss: 0.1930 - accuracy: 0.9420 - val_loss: 15.4088 - val_accuracy: 0.1568
Epoch 6/8
 - 6s - loss: 0.1755 - accuracy: 0.9485 - val_loss: 14.7107 - val_accuracy: 0.1607
Epoch 7/8
 - 6s - loss: 0.1615 - accuracy: 0.9518 - val_loss: 17.1745 - val_accuracy: 0.1644
Epoch 8/8
 - 6s - loss: 0.1494 - accuracy: 0.9547 - val_loss: 15.5428 - val_accuracy: 0.1553


In [0]:
y_test = np.argmax(y_test, 1)

predictions = model.predict(x_test)
predictions = np.argmax(predictions , 1)

In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
  
cm = confusion_matrix(y_test, predictions) 

print (f'Confusion Matrix : \n\n{cm}\n\n')
print (f'Accuracy Score : {accuracy_score(y_test, predictions)}')

Confusion Matrix : 

[[739   0   0   1   1   4  13   3  15   0   0   0   1   2   0   0   3   0
    4   4   6   3   0   0   0   0]
 [  0 765   0   4   0   1   0   0   0   1   3   1  11   2   1   0   1   1
    2   0   3   0   4   0   0   0]
 [  0   0 772   5   1   3   0   0   0   2   0   2   1   2   1   0   6   0
    1   0   1   3   0   0   0   0]
 [  0   2   0 785   0   0   2   0   1   3   0   0   0   0   0   0   0   2
    0   5   0   0   0   0   0   0]
 [  0   0   1   0 788  10   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0   0   0   0   0   0]
 [  2   1   2   1   6 777   0   0   0   5   0   0   0   2   2   1   0   0
    0   0   1   0   0   0   0   0]
 [  1   0   0   0   0   0 789   1   2   0   0   0   0   1   1   0   0   0
    0   2   3   0   0   0   0   0]
 [  0   0   1   1   0   2   2 782   5   1   0   0   0   0   0   0   0   0
    0   0   4   1   1   0   0   0]
 [ 23   0   0   3   0   1  14   1 733   1   0   0   0   1   0   0   0   3
    2   0   2   2   2  12   0   0